In [70]:
try:
    import tensorflow as tf
    from keras.models import Sequential, save_model, load_model
    from keras.layers import Dense, LSTM, Input, Dropout
    from keras.callbacks import EarlyStopping
    from padasip.filters import FilterRLS
    from sklearn.multioutput import MultiOutputRegressor
    from lightgbm import LGBMRegressor
    import pandas as pd
    import numpy as np
    from datetime import datetime, timedelta, timezone
    import json
    import pickle
    import os
    import sys
    import argparse
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    from DB_Utils import select_model_for_forecast, load_artifact_from_storage, store_forecasts, fetch_data
    from supabase import create_client, Client
    import traceback
    import plotly.express as px
    from dotenv import load_dotenv, find_dotenv

    os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"  # Or '2' or '3'

    KERAS_AVAILABLE = True
    PADASIP_AVAILABLE = True

    np.set_printoptions(suppress=True)

    load_dotenv()
    print("Env file found at location: ", find_dotenv())

except ImportError:
    print("TensorFlow/Keras not found. Keras models cannot be trained/saved natively.")
    KERAS_AVAILABLE = False
    # Define dummy classes if needed for type checking, though not strictly necessary here
    Sequential, save_model, load_model = object, lambda x, y: None, lambda x: None
    Dense, LSTM, Input, Dropout, EarlyStopping = object, object, object, object, object

    print("padasip not found. RLS filters cannot be loaded/used.")
    PADASIP_AVAILABLE = False
    FilterRLS = object


# --- Configuration & Constants --- (Same as before)
SUPABASE_URL = os.environ.get("SUPABASE_URL")
SUPABASE_KEY = os.environ.get("SUPABASE_SECRET_KEY")
if not SUPABASE_URL or not SUPABASE_KEY or "YOUR_SUPABASE_URL" in SUPABASE_URL:
    print("Error: SUPABASE_URL and SUPABASE_KEY environment variables must be set.")
    # sys.exit(1)

supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
DATA_SCHEMA = "data"
ML_SCHEMA = "ml"
STORAGE_BUCKET = "models"
FEEDER_ID_TO_TRAIN = 1
SCENARIO = "Day"  # Example
MODEL_VERSION = "v1.1_Is_Weekend"  # Updated version
TRAIN_START_DATE = "2024-01-01 00:00:00+00"
TRAIN_END_DATE = "2024-05-31 23:59:59+00"
VALIDATION_START_DATE = "2024-06-01 00:00:00+00"
VALIDATION_END_DATE = "2024-06-30 23:59:59+00"
DAY_HOURS = list(range(6, 20 + 1))
NIGHT_HOURS = list(range(0, 6)) + list(range(21, 24))
script_dir = os.path.dirname(os.path.abspath(__file__)) if "__file__" in locals() else os.getcwd()
TEMP_DIR = os.path.join(script_dir, "tmp")

Env file found at location:  h:\My Drive\Barbados_Forecasting_Tool_Final\.env


In [71]:
supabase.postgrest.schema(ML_SCHEMA)
lightgbm_results = supabase.table("forecasts").select("target_timestamp, forecast_value, actual_value").eq("feeder_id", 1).eq("model_id", 1).gte("target_timestamp", '2024-07-03').execute()
lstm_rls_results = supabase.table("forecasts").select("target_timestamp, forecast_value").eq("feeder_id", 1).eq("model_id", 19).gte("target_timestamp", '2024-07-03').execute()

lightgbm_results_df = pd.DataFrame(lightgbm_results.data).set_index("target_timestamp")
lstm_rls_results_df = pd.DataFrame(lstm_rls_results.data).set_index("target_timestamp")

lightgbm_results_df.rename(columns={"forecast_value": "lightgbm_forecast"}, inplace=True)
lstm_rls_results_df.rename(columns={"forecast_value": "lstm_rls_forecast"}, inplace=True)

px.line(pd.concat([lightgbm_results_df, lstm_rls_results_df], axis=1))
# select * from ml.forecasts where Feeder_ID = 1 and target_timestamp >= '2024-07-03' and model_id = 19

KeyError: "None of ['target_timestamp'] are in the columns"

In [ ]:
feeder_id = 1

feeder_2_lightgbm_baseline_24hr_model_id = supabase.table("models").select("model_id").eq("feeder_id", feeder_id).eq("model_architecture_type", "LightGBM_Baseline").eq("scenario_type", "24hr").execute().data[0]['model_id']
feeder_2_lstm_rls_combined_24hr_model_id = supabase.table("models").select("model_id").eq("feeder_id", feeder_id).eq("model_architecture_type", "LSTM_RLS_Combined").eq("scenario_type", "24hr").execute().data[0]['model_id']
feeder_2_lightgbm_baseline_day_model_id = supabase.table("models").select("model_id").eq("feeder_id", feeder_id).eq("model_architecture_type", "LightGBM_Baseline").eq("scenario_type", "Day").execute().data[0]['model_id']
feeder_2_lstm_rls_combined_day_model_id = supabase.table("models").select("model_id").eq("feeder_id", feeder_id).eq("model_architecture_type", "LSTM_RLS_Combined").eq("scenario_type", "Day").execute().data[0]['model_id']
feeder_2_lightgbm_baseline_night_model_id = supabase.table("models").select("model_id").eq("feeder_id", feeder_id).eq("model_architecture_type", "LightGBM_Baseline").eq("scenario_type", "Night").execute().data[0]['model_id']
feeder_2_lstm_rls_combined_night_model_id = supabase.table("models").select("model_id").eq("feeder_id", feeder_id).eq("model_architecture_type", "LSTM_RLS_Combined").eq("scenario_type", "Night").execute().data[0]['model_id']

feeder_2_lightgbm_baseline_24hr_model_id, feeder_2_lstm_rls_combined_24hr_model_id, feeder_2_lightgbm_baseline_day_model_id, feeder_2_lstm_rls_combined_day_model_id, feeder_2_lightgbm_baseline_night_model_id, feeder_2_lstm_rls_combined_night_model_id, 

(1, 19, 2, 20, 3, 21)

In [ ]:
feeder_2_lightgbm_baseline_24hr_model_results = supabase.table("forecasts").select("target_timestamp, forecast_value, actual_value").eq("feeder_id", feeder_id).eq("model_id", feeder_2_lightgbm_baseline_24hr_model_id).gte("target_timestamp", '2024-07-03').execute().data
feeder_2_lstm_rls_combined_24hr_model_results = supabase.table("forecasts").select("target_timestamp, forecast_value").eq("feeder_id", feeder_id).eq("model_id", feeder_2_lstm_rls_combined_24hr_model_id).gte("target_timestamp", '2024-07-03').execute().data
feeder_2_lightgbm_baseline_day_model_results = supabase.table("forecasts").select("target_timestamp, forecast_value").eq("feeder_id", feeder_id).eq("model_id", feeder_2_lightgbm_baseline_day_model_id).gte("target_timestamp", '2024-07-03').execute().data
feeder_2_lstm_rls_combined_day_model_results = supabase.table("forecasts").select("target_timestamp, forecast_value").eq("feeder_id", feeder_id).eq("model_id", feeder_2_lstm_rls_combined_day_model_id).gte("target_timestamp", '2024-07-03').execute().data
feeder_2_lightgbm_baseline_night_model_results = supabase.table("forecasts").select("target_timestamp, forecast_value").eq("feeder_id", feeder_id).eq("model_id", feeder_2_lightgbm_baseline_night_model_id).gte("target_timestamp", '2024-07-03').execute().data
feeder_2_lstm_rls_combined_night_model_results = supabase.table("forecasts").select("target_timestamp, forecast_value").eq("feeder_id", feeder_id).eq("model_id", feeder_2_lstm_rls_combined_night_model_id).gte("target_timestamp", '2024-07-03').execute().data

feeder_2_lightgbm_baseline_24hr_model_results = pd.DataFrame(feeder_2_lightgbm_baseline_24hr_model_results).set_index("target_timestamp")
feeder_2_lstm_rls_combined_24hr_model_results = pd.DataFrame(feeder_2_lstm_rls_combined_24hr_model_results).set_index("target_timestamp")
feeder_2_lightgbm_baseline_day_model_results = pd.DataFrame(feeder_2_lightgbm_baseline_day_model_results).set_index("target_timestamp")
feeder_2_lstm_rls_combined_day_model_results = pd.DataFrame(feeder_2_lstm_rls_combined_day_model_results).set_index("target_timestamp")
feeder_2_lightgbm_baseline_night_model_results = pd.DataFrame(feeder_2_lightgbm_baseline_night_model_results).set_index("target_timestamp")
feeder_2_lstm_rls_combined_night_model_results = pd.DataFrame(feeder_2_lstm_rls_combined_night_model_results).set_index("target_timestamp")

feeder_2_lightgbm_day_night_model_results = pd.concat([feeder_2_lightgbm_baseline_day_model_results, feeder_2_lightgbm_baseline_night_model_results], axis=0)
feeder_2_lstm_rls_combined_day_night_model_results = pd.concat([feeder_2_lstm_rls_combined_day_model_results, feeder_2_lstm_rls_combined_night_model_results], axis=0)


feeder_2_lightgbm_baseline_24hr_model_results.index = pd.to_datetime(feeder_2_lightgbm_baseline_24hr_model_results.index)
feeder_2_lstm_rls_combined_24hr_model_results.index = pd.to_datetime(feeder_2_lstm_rls_combined_24hr_model_results.index)
feeder_2_lightgbm_day_night_model_results.index = pd.to_datetime(feeder_2_lightgbm_day_night_model_results.index)
feeder_2_lstm_rls_combined_day_night_model_results.index = pd.to_datetime(feeder_2_lstm_rls_combined_day_night_model_results.index)


feeder_2_lightgbm_baseline_24hr_model_results.rename(columns={"forecast_value": "lightgbm_24hr_forecast"}, inplace=True)
feeder_2_lstm_rls_combined_24hr_model_results.rename(columns={"forecast_value": "lstm_rls_24hr_forecast"}, inplace=True)
feeder_2_lightgbm_day_night_model_results.rename(columns={"forecast_value": "lightgbm_day_night_forecast"}, inplace=True)
feeder_2_lstm_rls_combined_day_night_model_results.rename(columns={"forecast_value": "lstm_rls_day_night_forecast"}, inplace=True)
# feeder_2_lightgbm_baseline_night_model_results.rename(columns={"forecast_value": "lightgbm_night_forecast"}, inplace=True)
# feeder_2_lstm_rls_combined_night_model_results.rename(columns={"forecast_value": "lstm_rls_night_forecast"}, inplace=True)


all_results = pd.concat([feeder_2_lightgbm_baseline_24hr_model_results, feeder_2_lstm_rls_combined_24hr_model_results, feeder_2_lightgbm_day_night_model_results, feeder_2_lstm_rls_combined_day_night_model_results], axis=1)

px.line(all_results)


In [ ]:
feeder_id = 2

lstm_rls_combined_24hr_model_id = supabase.table("models").select("model_id").eq("feeder_id", feeder_id).eq("model_architecture_type", "LSTM_RLS_Combined").eq("scenario_type", "24hr").execute().data[-1]['model_id']
lstm_rls_combined_24hr_model_results = supabase.table("forecasts").select("target_timestamp, forecast_value, actual_value").eq("feeder_id", feeder_id).eq("model_id", lstm_rls_combined_24hr_model_id).execute()
lstm_rls_combined_24hr_model_results = pd.DataFrame(lstm_rls_combined_24hr_model_results.data).set_index("target_timestamp")

px.line(lstm_rls_combined_24hr_model_results)

In [136]:
feeder_id = 3

light_gbm_model_id = supabase.table("models").select("model_id").eq("feeder_id", feeder_id).eq("model_architecture_type", "LightGBM_Baseline").execute().data[-1]['model_id']
lstm_rls_model_id = supabase.table("models").select("model_id").eq("feeder_id", feeder_id).eq("model_architecture_type", "LSTM_RLS_Combined").execute().data[-1]['model_id']
lstm_change_model_id = supabase.table("models").select("model_id").eq("feeder_id", feeder_id).eq("model_architecture_type", "LSTM_Change_in_Load").execute().data[-1]['model_id']

def get_latest_model_forecast(feeder_id, model_id):
    latest_data = supabase.table("forecasts").select("target_timestamp, forecast_value, actual_value").eq("feeder_id", feeder_id).eq("model_id", model_id).execute().data
    latest_data = pd.DataFrame(latest_data).set_index("target_timestamp")
    latest_data.index = pd.to_datetime(latest_data.index)
    return latest_data

light_gbm_latest_data = get_latest_model_forecast(feeder_id, light_gbm_model_id).rename(columns={"forecast_value": "lightgbm_forecast"})
lstm_rls_latest_data = get_latest_model_forecast(feeder_id, lstm_rls_model_id)['forecast_value']
lstm_rls_latest_data.name = "lstm_rls_forecast"
lstm_change_latest_data = get_latest_model_forecast(feeder_id, lstm_change_model_id)['forecast_value']
lstm_change_latest_data.name = "lstm_change_forecast"


combined_results = pd.concat([light_gbm_latest_data, lstm_rls_latest_data, lstm_change_latest_data], axis=1)


# latest_data = supabase.table("forecasts").select("target_timestamp, forecast_value, actual_value").eq("feeder_id", feeder_id).eq("model_id", model_id).execute().data
# latest_data = pd.DataFrame(latest_data).set_index("target_timestamp")
# latest_data.index = pd.to_datetime(latest_data.index)

# px.line(combined_results)
px.line(combined_results)


In [125]:
combined_results.drop(columns='actual_value').columns

Index(['lightgbm_forecast', 'lstm_rls_forecast', 'lstm_change_forecast'], dtype='object')

In [129]:
from sklearn.metrics import mean_absolute_percentage_error, root_mean_squared_error

def smape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    diff = np.abs(y_true - y_pred) / denominator
    return np.mean(diff) * 100  # SMAPE as a percentage


for i in combined_results.drop(columns='actual_value').columns:
    print("Model: ", i)
    print("MAE: ", mean_absolute_error(combined_results['actual_value'], combined_results[i]))
    print("RMSE: ", root_mean_squared_error(combined_results['actual_value'], combined_results[i]))
    print("SMAPE: ", smape(combined_results['actual_value'], combined_results[i]))

Model:  lightgbm_forecast
MAE:  61.465886310562034
RMSE:  115.28101317311723
SMAPE:  7.409867925626114
Model:  lstm_rls_forecast
MAE:  59.135694009498714
RMSE:  111.6073763670486
SMAPE:  7.730234187098442
Model:  lstm_change_forecast


ValueError: Input contains NaN.